<a href="https://colab.research.google.com/github/sreeharsha243/BigData/blob/main/harsha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=d489965842701445020b99f840f51dd7caa72f4350e0b7215953c412e8cb509a
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [5]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadTextFile").getOrCreate()
text_file_path = r'/content/sample_data/lvl.txt'
text_data = spark.read.text(text_file_path)


text_data.show(truncate=False)

spark.stop()


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                  |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
#Removing blank lines, stop words and  punctuation characters before further processing

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')


text_file_path = r'/content/sample_data/lvl.txt'
with open(text_file_path, 'r') as file:
    text_content = file.read()


text_content = '\n'.join(line for line in text_content.splitlines() if line.strip())


tokens = word_tokenize(text_content)
filtered_tokens = [token for token in tokens if re.match(r'^\w+$', token)]


stop_words = set(stopwords.words('english'))
filtered_text = ' '.join(word for word in filtered_tokens if word.lower() not in stop_words)

print(filtered_text)


grew stronger day always listen world people around could brave enough stuck together means without love much would even go far lip glossing one bras someday lucky maybe even wear together four months already know one bring much happiness life imagine without love everything love anything world lucky side mean world hope spend every day making happy Every time look heart skips beat feel hugs strength becomes tenfold imagine day without certain want know feels love words even say light life soulmate best friend look fine skin soft love seeing every day get butterflies stomach anytime close liked one want forever ever guess trying say love much mean world trade anything heart soul million years never anyone else Love word introduced around needed apparent meant almost obnoxiously obvious hope time erase everything feeling bad love around beautiful trust telling truth love saying want Valentines day true Let make relationship status official case holdouts dating think going pretty well kn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#Word with highest number of occurrence

from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("RDDWordCount")
sc = SparkContext(conf=conf)
text_file_path = "/content/sample_data/lvl.txt"
text_rdd = sc.textFile(text_file_path)
word_counts = text_rdd.flatMap(lambda line: line.split()).countByValue()
most_common_word = max(word_counts, key=word_counts.get)
print("Word with highest occurrence:", most_common_word)
sc.stop()

Word with highest occurrence: I


In [11]:
#Character with highest number of occurrence
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("RDDCharacterCount")
sc = SparkContext(conf=conf)
text_file_path = "/content/sample_data/lvl.txt"
text_rdd = sc.textFile(text_file_path)
character_counts = text_rdd.flatMap(lambda line: list(line)).countByValue()
most_common_character = max(character_counts, key=character_counts.get)
print("Character with highest occurrence:", most_common_character)
sc.stop()

Character with highest occurrence:  


In [12]:
#Compute occurrence of any given word accepted from keyboard

from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("RDDWordOccurrence")
sc = SparkContext(conf=conf)
text_file_path = "/content/sample_data/lvl.txt"
text_rdd = sc.textFile(text_file_path)
target_word = input("Enter a word: ")
word_occurrences = text_rdd.filter(lambda line: target_word in line).count()
print("Occurrences of", target_word, ":", word_occurrences)
sc.stop()

Enter a word: hello
Occurrences of hello : 0


In [16]:
#Line sorted in descending order of number of words they contain

from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("RDDLineSorting")
sc = SparkContext.getOrCreate()
text_file_path = "/content/sample_data/lvl.txt"
text_rdd = sc.textFile(text_file_path)
lines_sorted = text_rdd.map(lambda line: (line, len(line.split()))).sortBy(lambda x: x[1], ascending=False)
sorted_lines = lines_sorted.map(lambda x: x[0]).collect()
for line in sorted_lines:
    print(line)
sc.stop()


I love you so much and I am so glad I met you. You are the one I have been waiting for all of my life. The little things, like making sure to let me know when I look beautiful and holding my hand whenever you can, makes me feel so loved. You are the love of my life. Without you I am incomplete. I love you more than any woman I have ever met or ever will meet in the future. You are amazing
Every time I look at you, my heart skips a beat. When I feel your hugs, my strength becomes tenfold. I cannot imagine a day without you — and am certain I don’t want to know what that feels like… I love you more than words can even say; you are the light of my life, my soulmate, and my best friend.
You are the love of my life; you are my world. I would do anything to make you happy. I want to see your beautiful smile every moment of every day. You bring me joy and happiness, and I promise to give you all of mine. You mean the world to me, and I will forever love you.
I love you and have never been so 

In [17]:
#Take 80% sample twice and perform intersection between them and  check number of lines common.


from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("RDDIntersection")
sc = SparkContext.getOrCreate()
text_file_path = "/content/sample_data/lvl.txt"
text_rdd = sc.textFile(text_file_path)
sample1 = text_rdd.sample(False, 0.8)
sample2 = text_rdd.sample(False, 0.8)
common_lines = sample1.intersection(sample2)
common_lines_count = common_lines.count()
print("Number of common lines:", common_lines_count)

sc.stop()

Number of common lines: 12


In [18]:
#Find the number of records whose income > 8000

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CarsDataFrame").getOrCreate()
csv_file_path = "/content/sample_data/cars - a.csv"
cars_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
# Perform the operation: Find the number of records with income > 8000
income_gt_8000_count = cars_df.filter(cars_df["income"] > 8000).count()
print("Number of records with income > 8000:", income_gt_8000_count)
spark.stop()


Number of records with income > 8000: 295


In [19]:
#Find the number of records whose income <4000 and sales>1000

from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("CarsDataFrame").getOrCreate()
csv_file_path = "/content/sample_data/cars - a.csv"
cars_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Perform the operation: Find the number of records with income < 4000 and sales > 1000
filtered_count = cars_df.filter((cars_df["income"] < 4000) & (cars_df["sales"] > 1000)).count()
print("Number of records with income < 4000 and sales > 1000:", filtered_count)


spark.stop()

Number of records with income < 4000 and sales > 1000: 235


In [20]:
#Find the number of records for different ages and order them in  descending order


from pyspark.sql import SparkSession
from pyspark.sql.functions import col


spark = SparkSession.builder.appName("CarsDataFrame").getOrCreate()
csv_file_path = "/content/sample_data/cars - a.csv"
cars_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
age_counts = cars_df.groupBy("age").count().orderBy(col("count").desc())
age_counts.show()
spark.stop()


+---+-----+
|age|count|
+---+-----+
| 25|   41|
| 30|   34|
| 27|   33|
| 37|   32|
| 24|   32|
| 28|   30|
| 36|   30|
| 26|   29|
| 29|   29|
| 21|   29|
| 22|   28|
| 20|   28|
| 23|   25|
| 40|   24|
| 49|   24|
| 45|   24|
| 59|   23|
| 46|   23|
| 48|   23|
| 58|   23|
+---+-----+
only showing top 20 rows



In [21]:
#Find mean sales of male(1) and female (0)

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg


spark = SparkSession.builder.appName("CarsDataFrame").getOrCreate()
csv_file_path = "/content/sample_data/cars - a.csv"
cars_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
mean_sales_by_gender = cars_df.groupBy("gender").agg(avg("sales").alias("mean_sales"))
mean_sales_by_gender.show()

spark.stop()

+------+------------------+
|gender|        mean_sales|
+------+------------------+
|     1|11371.726720647774|
|     0|12024.953091684434|
+------+------------------+



In [22]:
#Find the descriptive statistics of debt

from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("CarsDataFrame").getOrCreate()
csv_file_path = "/content/sample_data/cars - a.csv"
cars_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
debt_stats = cars_df.select("debt").summary()
debt_stats.show()

spark.stop()


+-------+------------------+
|summary|              debt|
+-------+------------------+
|  count|               963|
|   mean|14109.004153686397|
| stddev| 18273.70248104874|
|    min|                 0|
|    25%|              1472|
|    50%|              6236|
|    75%|             16771|
|    max|             59770|
+-------+------------------+



In [23]:
#Find correlation between miles and debt

from pyspark.sql import SparkSession
from pyspark.sql.functions import corr


spark = SparkSession.builder.appName("CarsDataFrame").getOrCreate()
csv_file_path = "/content/sample_data/cars - a.csv"
cars_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
correlation = cars_df.stat.corr("miles", "debt")
print("Correlation between miles and debt:", correlation)

spark.stop()


Correlation between miles and debt: 0.5447908747188632


In [24]:
#Convert the dataframe to table and write an SQL query for finding  grouping of age,gender and number of records of each group

from pyspark.sql import SparkSession
from pyspark.sql.functions import corr

spark = SparkSession.builder.appName("CarsDataFrame").getOrCreate()
csv_file_path = "/content/sample_data/cars - a.csv"
cars_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
correlation = cars_df.stat.corr("miles", "debt")
print("Correlation between miles and debt:", correlation)

spark.stop()


Correlation between miles and debt: 0.5447908747188632


In [25]:
#Convert the dataframe to table and write an SQL query for finding grouping of age,gender and number of records of each group

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CarsDataFrame").getOrCreate()
csv_file_path = "/content/sample_data/cars - a.csv"
cars_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
cars_df.createOrReplaceTempView("cars_table")
sql_query = """
SELECT age, gender, COUNT(*) AS record_count
FROM cars_table
GROUP BY age, gender
"""
grouped_records = spark.sql(sql_query)
grouped_records.show()

spark.stop()

+---+------+------------+
|age|gender|record_count|
+---+------+------------+
| 30|     0|          14|
| 58|     1|          12|
| 48|     1|           9|
| 54|     0|           3|
| 56|     0|           7|
| 25|     1|          20|
| 42|     0|           4|
| 41|     0|           7|
| 29|     0|          13|
| 36|     1|          10|
| 59|     0|          10|
| 23|     0|          10|
| 41|     1|           5|
| 26|     0|          16|
| 39|     0|           6|
| 28|     1|          16|
| 26|     1|          13|
| 31|     1|           7|
| 59|     1|          13|
| 46|     0|           9|
+---+------+------------+
only showing top 20 rows



In [29]:
#Store the results of question in a csv called carf.csv

from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("CarsDataFrame").getOrCreate()
csv_file_path = "/content/sample_data/cars - a.csv"
cars_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
cars_df.createOrReplaceTempView("cars_table")
sql_query = """
SELECT age, gender, COUNT(*) AS record_count
FROM cars_table
GROUP BY age, gender
"""
grouped_records = spark.sql(sql_query)
output_csv_path = "/content/saple_data/carf.csv"
grouped_records.write.csv(output_csv_path, header=True)


In [30]:
#Data preprocessing

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("AirQualityPreprocessing").getOrCreate()
csv_file_path = "/content/sample_data/airqualitydataset.csv"
airquality_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
valid_column_name = "PM2_5"  # Using underscore as a valid alternative
renamed_df = airquality_df.withColumnRenamed("PM2.5", valid_column_name)
print("Initial Schema:")
airquality_df.printSchema()
print("Sample Data:")
airquality_df.show(5)
print("Renamed Schema:")
renamed_df.printSchema()
print("Sample Data with Renamed Column:")
renamed_df.show(5)

# Data Preprocessing
# Drop rows with missing values
preprocessed_df = renamed_df.dropna()
selected_columns = [
    valid_column_name, "PM10", "NO", "NO2", "NOx", "NH3",
    "CO", "SO2", "O3", "Benzene", "Toluene", "Xylene", "AQI"
]
preprocessed_df = preprocessed_df.select(*selected_columns)
print("Preprocessed Schema:")
preprocessed_df.printSchema()
print("Preprocessed Sample Data:")
preprocessed_df.show(5)
spark.stop()


Initial Schema:
root
 |-- PM2.5: double (nullable = true)
 |-- PM10: double (nullable = true)
 |-- NO: double (nullable = true)
 |-- NO2: double (nullable = true)
 |-- NOx: double (nullable = true)
 |-- NH3: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- SO2: double (nullable = true)
 |-- O3: double (nullable = true)
 |-- Benzene: double (nullable = true)
 |-- Toluene: double (nullable = true)
 |-- Xylene: double (nullable = true)
 |-- AQI: integer (nullable = true)

Sample Data:
+-----+------+----+-----+-----+-----+----+-----+------+-------+-------+------+---+
|PM2.5|  PM10|  NO|  NO2|  NOx|  NH3|  CO|  SO2|    O3|Benzene|Toluene|Xylene|AQI|
+-----+------+----+-----+-----+-----+----+-----+------+-------+-------+------+---+
| 81.4| 124.5|1.44| 20.5|12.08|10.72|0.12|15.24|127.09|    0.2|    6.5|  0.06|184|
|78.32|129.06|1.26| 26.0|14.85|10.28|0.14|26.96|117.44|   0.22|   7.95|  0.08|197|
|88.76|135.32| 6.6|30.85|21.77|12.91|0.11|33.59|111.81|   0.29|   7.63|  0.12|198|


In [31]:
#Filter our important columns contributing to AQI

from pyspark.sql import SparkSession
from pyspark.sql.functions import col


spark = SparkSession.builder.appName("AirQualityFilterColumns").getOrCreate()
csv_file_path = "/content/sample_data/airqualitydataset.csv"
airquality_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
important_columns = [
    "PM2.5", "PM10", "NO", "NO2", "NOx", "NH3",
    "CO", "SO2", "O3", "Benzene", "Toluene", "Xylene", "AQI"
]
selected_columns = [col("`" + column + "`") for column in important_columns]
filtered_df = airquality_df.select(*selected_columns)
print("Filtered Schema:")
filtered_df.printSchema()
print("Filtered Sample Data:")
filtered_df.show(5)
spark.stop()


Filtered Schema:
root
 |-- PM2.5: double (nullable = true)
 |-- PM10: double (nullable = true)
 |-- NO: double (nullable = true)
 |-- NO2: double (nullable = true)
 |-- NOx: double (nullable = true)
 |-- NH3: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- SO2: double (nullable = true)
 |-- O3: double (nullable = true)
 |-- Benzene: double (nullable = true)
 |-- Toluene: double (nullable = true)
 |-- Xylene: double (nullable = true)
 |-- AQI: integer (nullable = true)

Filtered Sample Data:
+-----+------+----+-----+-----+-----+----+-----+------+-------+-------+------+---+
|PM2.5|  PM10|  NO|  NO2|  NOx|  NH3|  CO|  SO2|    O3|Benzene|Toluene|Xylene|AQI|
+-----+------+----+-----+-----+-----+----+-----+------+-------+-------+------+---+
| 81.4| 124.5|1.44| 20.5|12.08|10.72|0.12|15.24|127.09|    0.2|    6.5|  0.06|184|
|78.32|129.06|1.26| 26.0|14.85|10.28|0.14|26.96|117.44|   0.22|   7.95|  0.08|197|
|88.76|135.32| 6.6|30.85|21.77|12.91|0.11|33.59|111.81|   0.29|   7.63|  

In [33]:
#Split into 70, 30 train and test dataframes

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataSplitting").getOrCreate()
csv_file_path = "/content/sample_data/airqualitydataset.csv"
airquality_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
train_ratio = 0.7
test_ratio = 1.0 - train_ratio
train_data, test_data = airquality_df.randomSplit([train_ratio, test_ratio], seed=42)
print("Number of rows in train_data:", train_data.count())
print("Number of rows in test_data:", test_data.count())
spark.stop()

Number of rows in train_data: 7325
Number of rows in test_data: 2989


In [ ]:
#Apply Linear Regression, Decision Tree classifier and Gradient Boost models on train data frames

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.sql.functions import col
# Create a Spark session
spark = SparkSession.builder.appName("ModelTraining").getOrCreate()
csv_file_path = "/content/sample_data/airqualitydataset.csv"
airquality_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
feature_columns = ["PM2.5", "PM10", "NO", "NO2", "NOx", "NH3", "CO", "SO2", "O3", "Benzene", "Toluene", "Xylene"]
target_column = "AQI"
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
feature_vector_df = assembler.transform(airquality_df)
train_ratio = 0.7
test_ratio = 1.0 - train_ratio
train_data, test_data = feature_vector_df.randomSplit([train_ratio, test_ratio], seed=42)
linear_regression = LinearRegression(featuresCol="features", labelCol=target_column)
linear_model = linear_regression.fit(train_data)
decision_tree = DecisionTreeClassifier(featuresCol="features", labelCol=target_column)
dt_model = decision_tree.fit(train_data)
gradient_boost = GBTClassifier(featuresCol="features", labelCol=target_column)
gbt_model = gradient_boost.fit(train_data)
spark.stop()


In [ ]:
#For each model find predictions on test dataframes and evaluate prediction accuracy.

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier
spark = SparkSession.builder.appName("ModelEvaluation").getOrCreate()
csv_file_path = "/content/sample_data/airqualitydataset.csv"
airquality_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)


feature_columns = ["PM2.5", "PM10", "NO", "NO2", "NOx", "NH3", "CO", "SO2", "O3", "Benzene", "Toluene", "Xylene"]
target_column = "AQI"
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
feature_vector_df = assembler.transform(airquality_df)
train_ratio = 0.7
test_ratio = 1.0 - train_ratio
train_data, test_data = feature_vector_df.randomSplit([train_ratio, test_ratio], seed=42)
linear_regression = LinearRegression(featuresCol="features", labelCol=target_column)
linear_model = linear_regression.fit(train_data)
linear_predictions = linear_model.transform(test_data)
decision_tree = DecisionTreeClassifier(featuresCol="features", labelCol=target_column)
dt_model = decision_tree.fit(train_data)
dt_predictions = dt_model.transform(test_data)
gradient_boost = GBTClassifier(featuresCol="features", labelCol=target_column)
gbt_model = gradient_boost.fit(train_data)
gbt_predictions = gbt_model.transform(test_data)
regression_evaluator = RegressionEvaluator(labelCol=target_column)
linear_rmse = regression_evaluator.evaluate(linear_predictions, {regression_evaluator.metricName: "rmse"})
classification_evaluator = MulticlassClassificationEvaluator(labelCol=target_column)
dt_accuracy = classification_evaluator.evaluate(dt_predictions, {classification_evaluator.metricName: "accuracy"})
gbt_accuracy = classification_evaluator.evaluate(gbt_predictions, {classification_evaluator.metricName: "accuracy"})
print("Linear Regression RMSE:", linear_rmse)
print("Decision Tree Classifier Accuracy:", dt_accuracy)
print("Gradient Boosting Classifier Accuracy:", gbt_accuracy)
spark.stop()
